# process




## process group & session

**Process Group** is a collection of related processes which can all be signalled at once.

**Session** is a collection of process groups, which are either attached to a single terminal device(known as the controlling terminal) or not attached to any terminal.

Sessions are used for job control: one of the process groups in the session is the foreground process group, and can be sent signals by terminal control characters. You can think of a session with a controlling terminal as corresponding to a "login" on that terminal. (Daemons normally disassociate themselves from any controlling terminal by creating a new session without one.)

e.g. if you run some_app from the shell, the shell creates a new process group for it, and makes that the foreground process group of the session. (some_app might create some child processes; by default they will be part of the same process group.) If you then press ^Z, some_app's process group is signalled to stop it; and the shell's process group is switched to be the foreground process group again. Then e.g.bg %1 would start some_app's process group again, but keep it running in the background.


[Linux进程模型--会话和进程组](https://www.jianshu.com/p/d953cac76de9?utm_campaign=maleskine&utm_content=note&utm_medium=seo_notes&utm_source=recommendation)

[The process model](https://www.informit.com/articles/article.aspx?p=397655&seqNum=6)

-------------------

## thread_info and kernel stack

![](resources/process01.png)
![](resources/process02.png)
![](resources/process03.png)

### linux2.6/include/linux/sched.h

```c
union thread_union {
	struct thread_info thread_info;
	unsigned long stack[THREAD_SIZE/sizeof(long)];
};
```

### linux2.6/include/asm-i386/current.h
```c

static inline struct task_struct * get_current(void)
{
	return current_thread_info()->task;
}
 
#define current get_current()
```

### linux2.6/include/asm-i386/thread_info.h

```c
/*
 * low level task data that entry.S needs immediate access to
 * - this struct should fit entirely inside of one cache line
 * - this struct shares the supervisor stack pages
 * - if the contents of this structure are changed, the assembly constants must also be changed
 */
#ifndef __ASSEMBLY__

struct thread_info {
	struct task_struct	*task;		/* main task structure */
	struct exec_domain	*exec_domain;	/* execution domain */
	unsigned long		flags;		/* low level flags */
	unsigned long		status;		/* thread-synchronous flags */
	__u32			cpu;		/* current CPU */
	__s32			preempt_count; /* 0 => preemptable, <0 => BUG */


	mm_segment_t		addr_limit;	/* thread address space:
					 	   0-0xBFFFFFFF for user-thead
						   0-0xFFFFFFFF for kernel-thread
						*/
	struct restart_block    restart_block;

	unsigned long           previous_esp;   /* ESP of the previous stack in case
						   of nested (IRQ) stacks
						*/
	__u8			supervisor_stack[0];
};
```

1. `0-0XBFFFFFFF` = `0-3G`, user-thread only uses lower 3G address



```c
* how to get the thread information struct from C */
static inline struct thread_info *current_thread_info(void)
{
	struct thread_info *ti;
	__asm__("andl %%esp,%0; ":"=r" (ti) : "0" (~(THREAD_SIZE - 1)));
	return ti;
}

/* how to get the current stack pointer from C */
register unsigned long current_stack_pointer asm("esp") __attribute_used__;

/* thread information allocation */
#ifdef CONFIG_DEBUG_STACK_USAGE
#define alloc_thread_info(tsk)					\
	({							\
		struct thread_info *ret;			\
								\
		ret = kmalloc(THREAD_SIZE, GFP_KERNEL);		\
		if (ret)					\
			memset(ret, 0, THREAD_SIZE);		\
		ret;						\
	})
#else
#define alloc_thread_info(tsk) kmalloc(THREAD_SIZE, GFP_KERNEL)
#endif
```

1. `current` is use the `esp & ~(THREAD_SIZE-1)`

2. `thread_info` is allocated by `kmalloc`

------------------------------------

## NO_HZ

[NO_HZ](../../refs/NO_HZ.txt)

---------------------------------------


## schedule

![](resources/runqueue01.png)
![](resources/runqueue02.png)

### linux2.6/include/linux/sched.h

```c
/*
 * Priority of a process goes from 0..MAX_PRIO-1, valid RT
 * priority is 0..MAX_RT_PRIO-1, and SCHED_NORMAL tasks are
 * in the range MAX_RT_PRIO..MAX_PRIO-1. Priority values
 * are inverted: lower p->prio value means higher priority.
 *
 * The MAX_USER_RT_PRIO value allows the actual maximum
 * RT priority to be separate from the value exported to
 * user-space.  This allows kernel threads to set their
 * priority to a value higher than any user task. Note:
 * MAX_RT_PRIO must not be smaller than MAX_USER_RT_PRIO.
 */

#define MAX_USER_RT_PRIO	100
#define MAX_RT_PRIO		MAX_USER_RT_PRIO

#define MAX_PRIO		(MAX_RT_PRIO + 40)

#define rt_task(p)		(unlikely((p)->prio < MAX_RT_PRIO))
```

1. 0-99 is real-time priority, 100-139 is normal priority

-----------------------------------